In [1]:
from imutils.video import FPS
import numpy as np
import imutils
import dlib
import cv2

In [2]:
def isinside(l, p):
    poly = np.array(l,dtype=np.int32)
    poly_new = poly.reshape((-1,1,2))
    result1 = cv2.pointPolygonTest(poly_new, p, False)
    #cv2.polylines(medianFrame,[poly_new],isClosed=True,color=(0,255,0),thickness=10)
    #plt.imshow(medianFrame)
    if result1 == 1.0:
        return True
    else:
        return False

In [3]:
l_ofpoints = [[0, 0], [0, 300], [960, 350], [960, 0]]
l_ofpoints2 = [[0, 500], [0, 720], [960, 720], [960, 600]]
lists_out = [l_ofpoints, l_ofpoints2]

In [18]:
video_path = '../PycharmProjects/pythonProject/venv/COUNTING/Videos/pedestrians.avi'
cap = cv2.VideoCapture(video_path)
fgbg = cv2.createBackgroundSubtractorMOG2()
trackers = []
fps = FPS().start()
Icount = 0
while cap.isOpened:
    ret, img = cap.read()
    if ret == True:
        Icount += 1
        if Icount > 1:
            img = cv2.resize(img, (960,720))
            rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if len(trackers)==0:
                fgmask = fgbg.apply(img)
                kernel = np.ones((6,6), np.uint8)
                fgmask = cv2.erode(fgmask, kernel, iterations=1)
                fgmask = cv2.dilate(fgmask, kernel, iterations=1)
                (contours, hierarchy) = cv2.findContours(fgmask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                for index, contour in enumerate(contours):
                    (x, y, w, h) = cv2.boundingRect(contour)
                    M = cv2.moments(contour)
                    center = (int(M['m10']/M['m00']), int(M['m01']/M['m00']))
                    if any([isinside(list, center) for list in lists_out]):
                        continue
                    if w>30 and h>40:
                        (startX, startY, endX, endY) = (x, y, x+w, y+h)
                        if Icount>2:
                            t = dlib.correlation_tracker()
                            rect = dlib.rectangle(startX, startY, endX, endY)
                            t.start_track(rgb, rect)
                            trackers.append(t)
                            cv2.rectangle(img, (startX, startY), (endX, endY), (255, 0, 0), 1)
            else:
                for t in trackers:
                    if Icount>2:
                        t.update(rgb)
                        pos = t.get_position()
                        startX = int(pos.left())
                        startY = int(pos.top())
                        endX = int(pos.right())
                        endY = int(pos.bottom())
                        cv2.rectangle(img, (startX, startY), (endX, endY), (255, 0, 0), 1)

        cv2.imshow('frame', img)
        if cv2.waitKey(1) & 0xFF == ord('s'):
            break
        fps.update()
    else:
        break
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 